In [ ]:
!pip install flask flask-ngrok pyngrok

In [ ]:
!pip install rembg[cpu]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
auth_key = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
import os
os.system(f"ngrok authtoken '{auth_key}'")

0

In [ ]:
from flask import Flask, request, send_file
from rembg import remove
from PIL import Image
import io
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/check', methods=['GET'])
def check_api():
  return "API OK", 200

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return "No file part in the request", 400

    file = request.files['file']

    if file.filename == '':
        return "No selected file", 400

    if file:
        try:
            # Read the image file
            input_image_bytes = file.read()
            input_image = Image.open(io.BytesIO(input_image_bytes))

            # Remove the background
            output_image = remove(input_image)

            # Save the output image to a byte stream
            output_byte_stream = io.BytesIO()
            output_image.save(output_byte_stream, format="PNG")
            output_byte_stream.seek(0)

            # Return the processed image
            return send_file(output_byte_stream, mimetype='image/png')

        except Exception as e:
            return f"Error processing image: {e}", 500

ngrok_tunnel = ngrok.connect(5000)
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://897a97dd9366.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 07:01:03] "GET /check HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 07:01:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 07:05:44] "POST /upload HTTP/1.1" 200 -
